In [3]:
import os
import numpy as np
from time import sleep
from astropy.io import fits
from astropy.time import Time

In [4]:
def get_catalog(ra, dec, target, outpath, table='swiftmastr', oformat='fits'):

    # build the command
    cmd = (f'./browse_extract_wget.pl table={table} position={ra},{dec} '
           f'outfile={outpath}/{target}/{target}xrt.fits format={oformat} radius=10')

    print(cmd)
    os.system(cmd)

In [5]:
def get_files(target, outpath):

    cat = fits.getdata(f'{outpath}/{target}/{target}xrt.fits')

    if cat.shape[0] < 1:
        return

    for i in range(cat.shape[0]):
        obsid = cat['obsid'][i]
        t = Time(cat['start_time'][i], format='mjd')
        y, m = t.iso.split('-')[:2]

        wget = (f'wget -P {outpath}/{target} -q -nH --no-check-certificate --cut-dirs=5 '
                '-r -l0 -c -N -np -R "index*" -erobots=off --retr-symlinks '
                f'https://heasarc.gsfc.nasa.gov/FTP/swift/data/obs/{y}_{m}//'
                f'{obsid}/xrt/products/sw{obsid}xpc_ex.img.gz')

        print(wget)
        os.system(wget)

        wget = (f'wget -P {outpath}/{target} -q -nH --no-check-certificate --cut-dirs=5 '
                '-r -l0 -c -N -np -R "index*" -erobots=off --retr-symlinks '
                f'https://heasarc.gsfc.nasa.gov/FTP/swift/data/obs/{y}_{m}//'
                f'{obsid}/xrt/event/sw{obsid}xpcw3po_cl.evt.gz')

        print(wget)
        os.system(wget)

In [1]:
def load_PSZcatalog(unconf=False):
    from astropy.table import Table                                                       
    from numpy import append as npappend                                             

    datapath = f'{os.environ["HOME"]}/Projects/planckClusters/catalogs'
    
    ps1 = Table.read(f'{datapath}/PSZ1v2.1.fits')
    ps2 = Table.read(f'{datapath}/PSZ2v1.fits')

    # convert to pandas
    df1 = ps1.to_pandas()
    df2 = ps2.to_pandas()

    if unconf:
        # only get unconfirmed sources                                                   
        df1 = df1.loc[df1['VALIDATION'] <= 3]                                         
        df2 = df2.loc[df2['VALIDATION'] == -1]  
    
    # clean up strings -- not required
    df1 = df1.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
    df2 = df2.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)

    # merge the catalogs together
    df_m = df1.merge(df2, how='outer', left_on='INDEX', right_on='PSZ', suffixes=('_PSZ1', '_PSZ2'))
    
    # get the columns that we want
    cols = df_m.columns[[0, 1, 4, 5, 8, 29, 33, 34, 37, 38, 40, 51]]
    df_final = df_m[cols]

    # remerge to find bits that were missing                                        
    df_final_bigger = df_final.merge(df2, how='left', left_on='INDEX_PSZ1',         
                                 right_on='PSZ')
    # fill in nans                                                                  
    for col in ['NAME', 'RA', 'DEC', 'SNR', 'REDSHIFT', 'INDEX']:                   
        df_final_bigger[col+'_PSZ2'] = df_final_bigger[col+'_PSZ2'].fillna(df_final_bigger[col])
    # fill in nans                                                                  
    for col in ['NAME', 'RA', 'DEC', 'SNR', 'REDSHIFT', 'INDEX']:
        df_final_bigger[col+'_PSZ2'] = df_final_bigger[col+'_PSZ2'].fillna(df_final_bigger[col])
    for col in ['NAME', 'RA', 'DEC']:
        df_final_bigger[col] = df_final_bigger[col+'_PSZ2'].fillna(df_final_bigger[col+'_PSZ1'])

    df_final_bigger = df_final_bigger[npappend(df_final_bigger.columns[:12].values, ['NAME', 'RA', 'DEC'])]

    return df_final_bigger
  

In [7]:
def write_info(outpath, idx, data):
    target = data.iloc[idx]['NAME'].replace(' ', '_')
    cols = data.columns.tolist()
    vals = data.iloc[idx]
    # convert the values into all strings
    vals_str = [f'{i:.8}' if isinstance(i, float) else i for i in vals.values.tolist()]
    with open(f'{outpath}/{target}/{target}.info', 'w') as f:
        f.write(f'{",".join(cols)}\n')
        f.write(f'{",".join(vals_str)}\n')        
    

In [73]:
%%capture

# get file data
data = load_PSZcatalog()
data = data.sort_index(axis=1)

outpath = './data_full'

for i, (ra, dec, name) in enumerate(zip(data['RA'], data['DEC'], data['NAME'])):

    print(name)
    name = name.replace(' ', '_')

    if not os.path.isdir(f'{outpath}/{name}'):
        os.makedirs(f'{outpath}/{name}')

    get_catalog(ra, dec, f'{name}', outpath)
    write_info(outpath, i, data)
    get_files(f'{name}', outpath)
    
    sleep(1)

In [8]:
data = load_PSZcatalog()

In [10]:
data['NAME']

0       PSZ2 G000.04+45.13
1       PSZ2 G000.40-41.86
2       PSZ2 G000.77-35.69
3       PSZ1 G001.00+25.71
4       PSZ2 G002.08-68.28
5       PSZ2 G002.77-56.16
6       PSZ2 G002.82+39.23
7       PSZ1 G003.09-22.51
8       PSZ1 G003.60-24.45
9       PSZ2 G003.93-59.41
10      PSZ2 G003.91-42.03
11      PSZ2 G004.13+56.81
12      PSZ1 G004.30+53.93
13      PSZ2 G004.45-19.55
14      PSZ2 G005.91-28.26
15      PSZ2 G006.05+29.43
16      PSZ2 G006.49+50.56
17      PSZ2 G006.68-35.55
18      PSZ2 G006.76+30.45
19      PSZ2 G006.82-34.71
20      PSZ2 G007.57-33.90
21      PSZ2 G007.76-36.83
22      PSZ2 G008.31-64.74
23      PSZ2 G008.47-56.34
24      PSZ2 G008.80-35.18
25      PSZ2 G008.94-81.22
26      PSZ2 G009.45-56.93
27      PSZ1 G011.20-40.36
28      PSZ2 G011.36-72.93
29      PSZ1 G011.50-40.11
               ...        
1913    PSZ2 G342.32+23.51
1914    PSZ2 G342.45+24.14
1915    PSZ2 G342.62-39.60
1916    PSZ2 G343.33+83.19
1917    PSZ2 G343.46+52.65
1918    PSZ2 G343.68+24.10
1